In [11]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

url = f'/Users/watsonjung/Documents/playdata/94.Streamlit/Data_Train.csv'
df = pd.read_csv(url)

def preprocessing(df):
    ### 1. Route Drop 처리
    df.drop('Route', axis=1, inplace=True)

    ### 2. Duration 컬럼을 '시간'과 '분' 단위로 분할 후 Duration 컬럼 drop
    df['Dep_Time'] = pd.to_datetime(df['Dep_Time'], format= '%H:%M').dt.time
    df['Duration_hour'] = df.Duration.str.extract('(\d+)h')
    df['Duration_min'] = df.Duration.str.extract('(\d+)m').fillna(0)
    df.drop('Duration', axis=1, inplace=True)
    df.drop(index=6474,inplace=True)

    df.Duration_hour = df.Duration_hour.astype('int64')
    df.Duration_min = df.Duration_min.astype('int64')
    df.Duration_hour = df.Duration_hour*60
    df['Duration_total'] = df.Duration_hour+df.Duration_min
    df.drop(columns=['Duration_hour','Duration_min','Arrival_Time'],inplace=True)

    ### 3. Airline 전처리
    air_count = df.Airline.value_counts().index
    airlist = [l for l in air_count if list(df.Airline).count(l) < 200]
    df.Airline = df.Airline.replace(airlist, 'Others')

    for t in range(len(air_count)):
        df.loc[df.Airline == air_count[t], 'Air_col'] = t
    df.drop(columns=['Airline'],inplace=True)

    ### 4. Additional_Info 전처리
    add_count = df.Additional_Info.value_counts().index
    additional_thing = [l for l in add_count if list(df.Additional_Info).count(l) < 20]
    df.Additional_Info = df.Additional_Info.replace(additional_thing, 'Others')

    add_count = df.Additional_Info.value_counts().index
    for t in range(len(add_count)):
        df.loc[df.Additional_Info == add_count[t], 'Add_col'] = t

    ### 5. Total_Stops 전처리
    df.loc[df.Total_Stops.isna(),'Total_Stops'] = '1 stop'

    def handle_stops(x):
        if x == 'non-stop': return 0
        return int(x.split()[0])

    df.Total_Stops = df.Total_Stops.apply(handle_stops)

    ### 6. Date_of_Journey 전처리
    df['Date_of_journey_DT'] = pd.to_datetime(df['Date_of_Journey'])
    df['weekday'] = pd.to_datetime(df['Date_of_journey_DT']).dt.weekday
    df['weekday_name'] = pd.to_datetime(df['Date_of_journey_DT']).dt.day_name()

    ### 7. Dep_Time 데이터 전처리
    df.Dep_Time = df.Dep_Time.astype(str)
    df['Dep_hour'] = df.Dep_Time.str.extract('([0-9]+)\:')
    df.drop(columns=['Dep_Time'],inplace=True)

    ### 8. 불필요 컬럼 drop
    df.drop(columns=['Date_of_Journey',
                     'Source','Destination',
                     'Date_of_journey_DT',
                     'Additional_Info',
                     'weekday'],inplace=True)

    ### 9.범주형 변수 처리
    df = pd.get_dummies(df, columns=['weekday_name','Add_col','Air_col'],drop_first=True)
df.head()

,Total_Stops,Price,Duration_total,Dep_hour,weekday_name_Monday,weekday_name_Saturday,weekday_name_Sunday,weekday_name_Thursday,weekday_name_Tuesday,weekday_name_Wednesday,Add_col_1.0,Add_col_2.0,Add_col_3.0,Air_col_1.0,Air_col_2.0,Air_col_3.0,Air_col_4.0,Air_col_5.0,Air_col_6.0
0,0,3897,170,22,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
1,2,7662,445,05,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,2,13882,1140,09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,6218,325,18,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,1,13302,285,16,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

url = f'/Users/watsonjung/Documents/playdata/94.Streamlit/Data_Train.csv'
df = pd.read_csv(url)

In [2]:
### 1. Route Drop 처리
# Total_Stops 와 Route가 관련 있는 컬럼인가
# df.Total_Stops.value_counts()
# df[df.Total_Stops == 'non-stop']
# df[df.Total_Stops == '1 stop']
# df[df.Total_Stops == '2 stops']
# df[df.Total_Stops == '3 stops']
df[df.Total_Stops == '4 stops']
# => stop 수에 맞춰 Route가 늘어나는 것 확인

# df.head()를 통해 불필요한 컬럼 및 데이터 전처리
# 가장 먼저 Route를 drop 처리 => Total_Stops 라는 컬럼으로 활용
df.drop('Route', axis=1, inplace=True)

In [3]:
### 2. Duration 컬럼을 '시간'과 '분' 단위로 분할 후 Duration 컬럼 drop
df['Dep_Time'] = pd.to_datetime(df['Dep_Time'], format= '%H:%M').dt.time
df['Duration_hour'] = df.Duration.str.extract('(\d+)h')
df['Duration_min'] = df.Duration.str.extract('(\d+)m').fillna(0)
df.drop('Duration', axis=1, inplace=True)

# Duration 에서 이상치 drop
# Total-stop이 2 stops 인데 5분 탑승으로 되어있어 drop
df.drop(index=6474,inplace=True)

# 계산을 위해 str -> int64 로 변경하고 Duration_hour을 분으로 변경
# 최종적으로 Duration_total 컬럼으로 내린다
df.Duration_hour = df.Duration_hour.astype('int64')
df.Duration_min = df.Duration_min.astype('int64')
df.Duration_hour = df.Duration_hour*60
df['Duration_total'] = df.Duration_hour+df.Duration_min

# 계산 과정에 사용된 컬럼들 및 사용하지 않을 Arrival_Time 컬럼 drop 처리
df.drop(columns=['Duration_hour','Duration_min','Arrival_Time'],inplace=True)

In [4]:
### 3. Airline 전처리
# count값에 맞춰 index 넘버링하기
air_count = df.Airline.value_counts().index

# 200 보다 적은 수의 airline은 Others 로 변환
airlist = [l for l in air_count if list(df.Airline).count(l) < 200]
df.Airline = df.Airline.replace(airlist, 'Others')

# Air_col : Airline을 번호로 분류
for t in range(len(air_count)):
    df.loc[df.Airline == air_count[t], 'Air_col'] = t
    
# 전처리 완료된 Airline 컬럼 drop
df.drop(columns=['Airline'],inplace=True)

In [5]:
### 4. Additional_Info 전처리
# 20 보다 적은 수의 Additional_Info Others 로 변환
add_count = df.Additional_Info.value_counts().index
additional_thing = [l for l in add_count if list(df.Additional_Info).count(l) < 20]
df.Additional_Info = df.Additional_Info.replace(additional_thing, 'Others')

# 건 수가 많은 순서로 변수로 지정
add_count = df.Additional_Info.value_counts().index

# Add_col 컬럼에 인덱스 번호로 넣기
for t in range(len(add_count)):
    df.loc[df.Additional_Info == add_count[t], 'Add_col'] = t

In [6]:
### 5. Total_Stops 전처리
# Total_Stops 결측값 1개 존재
# 동일 비행기는 없으며 같은 루트로 같은 시간에 운행 => 1 stop : 14건, 2 stops : 12건
# 결론 : 동일 루트 상에 비교 불가능하므로 전체적으로 봤을 때 1 stop이 절반 이상이므로 1 stop 으로 처리
df.loc[df.Total_Stops.isna(),'Total_Stops'] = '1 stop'

# 각 stop의 숫자만 뽑아 저장하기
# non-stop은 0으로 처리
def handle_stops(x):
    # if type(x) == float: return np.NaN
    if x == 'non-stop': return 0
    return int(x.split()[0])

df.Total_Stops = df.Total_Stops.apply(handle_stops)

In [7]:
### 6. Date_of_Journey 전처리
# object 형식으로 되어 있는 날짜들을 to_datetime 로 처리해서 날짜형식으로 변경 후 요일 추출
df['Date_of_journey_DT'] = pd.to_datetime(df['Date_of_Journey'])
df['weekday'] = pd.to_datetime(df['Date_of_journey_DT']).dt.weekday
df['weekday_name'] = pd.to_datetime(df['Date_of_journey_DT']).dt.day_name()

In [8]:
### 7. Dep_Time 데이터 전처리
# 시간 단위만 뽑아내기 위해 str 형식으로 변환
df.Dep_Time = df.Dep_Time.astype(str)

# 출발시간, 도착시간에서 시간 단위만 뽑고 Dep_Time 컬럼 drop 처리
df['Dep_hour'] = df.Dep_Time.str.extract('([0-9]+)\:')
df.drop(columns=['Dep_Time'],inplace=True)

In [9]:
### 8. 불필요 컬럼 drop
df.drop(columns=['Date_of_Journey','Source','Destination','Date_of_journey_DT','Additional_Info','weekday'],inplace=True)

,Total_Stops,Price,Duration_total,Air_col,Add_col,weekday_name,Dep_hour
0,0,3897,170,1.0,0.0,Sunday,22
1,2,7662,445,2.0,0.0,Saturday,05
2,2,13882,1140,0.0,0.0,Friday,09
3,1,6218,325,1.0,0.0,Thursday,18
4,1,13302,285,1.0,0.0,Thursday,16


In [10]:
### 9.범주형 변수 처리
df = pd.get_dummies(df, columns=['weekday_name','Add_col','Air_col'],drop_first=True)
df.head()

,Total_Stops,Price,Duration_total,Dep_hour,weekday_name_Monday,weekday_name_Saturday,weekday_name_Sunday,weekday_name_Thursday,weekday_name_Tuesday,weekday_name_Wednesday,Add_col_1.0,Add_col_2.0,Add_col_3.0,Air_col_1.0,Air_col_2.0,Air_col_3.0,Air_col_4.0,Air_col_5.0,Air_col_6.0
0,0,3897,170,22,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
1,2,7662,445,05,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,2,13882,1140,09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,6218,325,18,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,1,13302,285,16,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
